In [1]:
import pandas as pd
import pymysql.cursors
import numpy as np

In [2]:
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='054867', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)

In [3]:
try:

   with conn.cursor() as curs:
      sql="SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)

finally:

   conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
243       18.78  3.00  Female     No  Thur  Dinner     2
244       16.99  1.01  Female     No   Sun  Dinner     2
245       26.88  3.12    Male     No        Dinner     4
246       26.88  3.12    Male     No        Dinner     4
247       26.88  3.12    Male     No        Dinner     5

[248 rows x 7 columns]


In [4]:
df['tip_rate']=df.tip/df.total_bill*100

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [8]:
df.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
243,18.78,3.00,Female,No,Thur,Dinner,2,15.974441
244,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
245,26.88,3.12,Male,No,,Dinner,4,11.607143
246,26.88,3.12,Male,No,,Dinner,4,11.607143
247,26.88,3.12,Male,No,,Dinner,5,11.607143


In [10]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
tip_rate      0
dtype: int64

In [13]:
df.replace('',np.nan,inplace=True)

In [14]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           3
time          0
size          0
tip_rate      0
dtype: int64

In [18]:
df.sex.replace({'Female':0,'Male':1},inplace=True)
df.smoker.replace({'No':0,'Yes':1},inplace=True)
df.day.replace({'Thur':0,'Fri':1,'Sat':2,'Sun':3},inplace=True)
df.time.replace({'Lunch':0,'Dinner':1},inplace=True)

In [19]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,0,0,3.0,1,2,5.944673
1,10.34,1.66,1,0,3.0,1,3,16.054159
2,21.01,3.50,1,0,3.0,1,3,16.658734
3,23.68,3.31,1,0,3.0,1,2,13.978041
4,24.59,3.61,0,0,3.0,1,4,14.680765


In [20]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           3
time          0
size          0
tip_rate      0
dtype: int64

In [21]:
type(df)

pandas.core.frame.DataFrame

In [22]:
from sklearn.impute import SimpleImputer

In [23]:
data=df.values

In [25]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,248.000000,248.000000,248.000000,248.000000,245.000000,248.000000,248.000000,248.000000
mean,19.860484,2.991734,0.645161,0.375000,1.730612,0.725806,2.588710,15.985279
std,8.866048,1.378257,0.479432,0.485102,1.156274,0.447009,0.965075,6.110974
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.563814
25%,13.385000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,12.534046
50%,17.815000,2.960000,1.000000,0.000000,2.000000,1.000000,2.000000,15.384615
75%,24.527500,3.520000,1.000000,1.000000,3.000000,1.000000,3.000000,19.065829
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,71.034483


In [30]:
sum(np.isnan(data))

array([0, 0, 0, 0, 3, 0, 0, 0])

#### flatten은 위의 array를 밑에와 같이 만든다.

In [32]:
sum(np.isnan(data).flatten())

3

SimpleImputer(
    *,
    missing_values=nan,
    strategy='mean',
    fill_value=None,
    verbose=0,
    copy=True,
    add_indicator=False,
)

In [33]:
imputer =SimpleImputer(strategy='median')

In [34]:
# fit on the dataset
imputer.fit(data)

SimpleImputer(strategy='median')

In [39]:
data_trans=imputer.transform(data)

In [41]:
df_trans = pd.DataFrame(data_trans)

In [42]:
df_trans.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64

In [43]:
from sklearn.preprocessing import StandardScaler

StandardScaler
(*, copy=True, with_mean=True, with_std=True)

In [44]:
sc=StandardScaler()

In [46]:
sc_df=sc.fit_transform(data_trans)

In [47]:
df_transform_sc=pd.DataFrame(sc_df)

In [48]:
df_transform_sc

,0,1,2,3,4,5,6,7
0,-0.324416,-1.440762,-1.34840,-0.774597,1.103583,0.614636,-0.611248,-1.646368
1,-1.075985,-0.968199,0.74162,-0.774597,1.103583,0.614636,0.427036,0.011294
2,0.129916,0.369520,0.74162,-0.774597,1.103583,0.614636,0.427036,0.110427
3,0.431674,0.231386,0.74162,-0.774597,1.103583,0.614636,-0.611248,-0.329129
4,0.534520,0.449493,-1.34840,-0.774597,1.103583,0.614636,1.465320,-0.213902
...,...,...,...,...,...,...,...,...
243,-0.122114,0.006010,-1.34840,-0.774597,-1.511277,0.614636,-0.611248,-0.001777
244,-0.324416,-1.440762,-1.34840,-0.774597,1.103583,0.614636,-0.611248,-1.646368
245,0.793331,0.093252,0.74162,-0.774597,0.231963,0.614636,1.465320,-0.717887
246,0.793331,0.093252,0.74162,-0.774597,0.231963,0.614636,1.465320,-0.717887


In [49]:
from sklearn.decomposition import PCA

In [52]:
trans = PCA(n_components=4)
data_dim = trans.fit_transform(df_transform_sc)

In [53]:
trans.fit(data_dim)

PCA(n_components=4)

In [68]:
pca=trans.explained_variance_ratio_

In [69]:
sum(pca[:3])

0.839425664539384